# Example Usage of the package AntiIdClosedGroup

This package directly depends on
- [ClosedGroupFunctions.jl](https://github.com/fhoeddinghaus/ClosedGroupFunctions.jl.git)
- [Zd_Arithmetics.jl](https://github.com/fhoeddinghaus/Zd_Arithmetics.jl.git)

## Install the Package

    ] add git@github.com:Fhoeddinghaus/AntiIdClosedGroup.jl.git

or

    ] add https://github.com/Fhoeddinghaus/AntiIdClosedGroup.jl.git

## Load the Package

In [1]:
using ClosedGroupFunctions
using Zd_Arithmetics

import Zd_Arithmetics: ℤ₂

push!(LOAD_PATH,"../src/") # for developement, load the local package
using AntiIdClosedGroup

┌ Info: Precompiling AntiIdClosedGroup [top-level]
└ @ Base loading.jl:1317


## Generating the group

### Defining the parameters

In [2]:
n = 5 # size of the outer matrix
k = 4 # size of the anti-identity sub-block

4

### The standard generators


In [3]:
Ω, all_pos, all_generators = construct_generators(n,k);

There are Ω = (5 choose 4) = 5  possible basic permutations.


#### Outputting the generators as LaTeX

The generators can be converted to LaTeX-code if needed using

    po2matrix(...)
or

    all_pos2matrix(...)

(See [documentation](https://fhoeddinghaus.github.io/AntiIdClosedGroup.jl/).)

### Generating the groups using the `ClosedGroupFunctions.jl`-Package

In [74]:
group, num_all_multiplications = group_generator_basic(all_generators; prnt=false, commutes=false);

level #1 size 5 and 21 new elements (25 multiplications)
level #2 size 26 and 240 new elements (651 multiplications)
level #3 size 266 and 454 new elements (70080 multiplications)
level #4 size 720 and 0 new elements (447644 multiplications)

Ended after 4 iterations. 
The resulting group has 720 elements.


It's not necessary to use the generators, as the `group_generator_basic` takes all kind of elements. Someone may try smaller sets of generators, as seen below (permutations).

It's advised to save the calculated group to disk for later use by using `ClosedGroupFunctions.jl`'s `store_group(...)` and `load_group(...)` functions.

In [76]:
# store the group to disk
store_group("n$n" * "k$k", group)

In [87]:
# load the group from disk
group = load_group("n$n" * "k$k");

720

#### Labelling the group

In [96]:
# label the generators
all_generators_labelled = label_generators(all_generators);

# label the group with the fast method
#number_of_elements = length(group)
#labelled_group = labelled_group_generator_simple(all_generators_labelled, number_of_elements; commutes=false);

# or label the group with the shortest possible label (slower method)
labelled_group = labelled_group_generator_shortest(all_generators_labelled; prnt=false, commutes=false)[1];

level #1 size 5 and 21 new elements (25 multiplications)
level #2 size 26 and 240 new elements (651 multiplications)
level #3 size 266 and 454 new elements (70080 multiplications)
level #4 size 720 and 0 new elements (447644 multiplications)

Ended after 4 iterations. 
The resulting group has 720 elements.


## Investigation and Analysis

We now want to investigate the group and it's generators further.

### Permutations

To investigate the group/generators under permutations, we first have to label the elements like above and then calculate all possible permutations.

In [81]:
all_generators_labelled = label_generators(all_generators);

In [82]:
# all permutations of [1, 2, 3, 4, 5, 6, 7] can be found with Combinatorics.jl
# either by
# using Combinatorics
# collect(permutations(1:n))
# or without importing (AntiIdClosedGroup depends directly on Combinatorics.jl)
all_permutations = collect(AntiIdClosedGroup.permutations(1:n));

Now we can apply the permutations (or a some of them) to the labelled generators by using the following function. The output is redirected and **appended** into a file with a given prefix (`"permutation_cycles_"`) and the identifier (`"tmp_1"`), because it's quite long.

In [84]:
print_permutation_cycles(all_generators_labelled, all_permutations, "n$n" * "k$k")

After some analysis of the file, we find a list of permutations, that can construct the other generators from each other. In the file, `p[i]` corresponds to the `i`th permutation in the entered list, `all_permutations`.

In [35]:
# in this case, only one permutation is needed to calculate all other generators from one of them.
selected_permutations = [97] 
ps = all_permutations[selected_permutations]

Pₛ = [perm_rows(id(n), ps[i]) for i in 1:length(ps)]
Qₛ = [perm_cols(id(n), ps[i]) for i in 1:length(ps)] # Q = P_inv

# new set of generators
a = all_generators[1]
new_generators = [a, Pₛ..., Qₛ...];

In [ ]:
# calculate the group with the new generators

In [36]:
group_with_perms, num_all_multiplications_with_perms = group_generator_basic(new_generators; prnt=false, commutes=false);

level #1 size 3 and 7 new elements (9 multiplications)
level #2 size 10 and 28 new elements (91 multiplications)
level #3 size 38 and 296 new elements (1344 multiplications)
level #4 size 334 and 386 new elements (110112 multiplications)
level #5 size 720 and 0 new elements (406844 multiplications)

Ended after 5 iterations. 
The resulting group has 720 elements.


In [42]:
# As explained in the thesis results, the group stays the same under permutations, 
# because all possible permutations are already part of the group.
# This is therefore a method to possibly find a smaller set of generators.
group == group_with_perms

true

### Invariants and Conjugacy Classes

#### Calculating the conjugacy classes

In [100]:
conjugacy_classes = calculate_conjugacy_classes(labelled_group)


LoadError: The inverse of a sparse matrix can often be dense and can cause the computer to run out of memory. If you are sure you have enough memory, please convert your matrix to a dense matrix, e.g. by calling `Matrix`.

#### Invariants
The package provides a few invariants that can be used:

1. Order of an element: `ord(g)`
2. Trace of an element (with higher powers): `tr_of_power(g, power) = tr(g^power)`
3. Rank of an element (+ Id): `rank_of_power_plus_id(g, power, n) = rank(g^power + id(n))`
4. Generalized Arf invariant of an element: `generalized_arf(g, n; readable = true)`

To use this functions for investigation of the conjugacy classes, we can use the functions `apply_invariant_to_first_in_class(...)` or `apply_invariant_to_first_in_all_classes(...)` from the `ClosedGroupFunctions.jl` package.

Both of these functions expect an `invariant` with exactly one argument: the element `g`. Therefore we have to define wrappers for the invariants, that we want to look at.

In [127]:
inv_wrapper(labelled_group["ab"])

5×5 Matrix{ℤd{Bool, 2}}:
 0  1  0  0  0
 0  0  1  1  1
 1  0  0  1  1
 1  0  1  0  1
 1  0  1  1  0

In [114]:
inv_wrapper(A::AntiIdClosedGroup.SparseMatrixCSC{ℤ₂}) = inv(Matrix{ℤ₂}(A))

inv_wrapper (generic function with 3 methods)